In [248]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import warnings
#!pip install plotly_express
import plotly_express as px

In [249]:
#sign in plotly
py.sign_in("cicichen0407", "fWbXlSXEcpZKnnI0pJ7Z")
# set all the data to 2 decimal
pd.set_option("display.float.format", lambda x: "%.2f" % x)
#ignore warning message 
warnings.filterwarnings("ignore")

# World inbound toursits from other country 

In [250]:
#import inbound tourist data set
inbound_tourists = pd.read_excel("travel_data.xls", sheet_name="number_of_arrival", header=3, index_col=0)
inbound_tourists.head()

,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Country Name,,,,,,,,,,,,,,,,,,,,,
World,WLD,"International tourism, number of arrivals",ST.INT.ARVL,nan,nan,nan,nan,nan,nan,nan,...,897533847.97,956372259.67,997555261.48,1054602152.04,1106732071.67,1159450156.34,1206215744.42,1250467517.26,1341456973.64,nan
High income,HIC,"International tourism, number of arrivals",ST.INT.ARVL,nan,nan,nan,nan,nan,nan,nan,...,548395233.84,579418921.50,605848548.04,632071775.74,663569908.73,705386216.47,738643409.80,775115157.74,821295509.07,nan
OECD members,OED,"International tourism, number of arrivals",ST.INT.ARVL,nan,nan,nan,nan,nan,nan,nan,...,501640000.00,521680000.00,545475000.00,563725000.00,592155300.00,630272600.00,664177000.00,687370000.00,738749000.00,nan
Post-demographic dividend,PST,"International tourism, number of arrivals",ST.INT.ARVL,nan,nan,nan,nan,nan,nan,nan,...,494445000.00,518743300.00,542860000.00,569186900.00,598775800.00,619423900.00,647527600.00,686263000.00,731127200.00,nan
Europe & Central Asia,ECS,"International tourism, number of arrivals",ST.INT.ARVL,nan,nan,nan,nan,nan,nan,nan,...,468111994.97,480591397.95,512778373.29,538631437.74,567722575.23,579784449.31,603880514.15,615979433.54,669544120.13,nan


In [251]:
#import world country list data
country_list = pd.read_excel("travel_data.xls", sheet_name="country code")
country_list.head()

,Country Name,Country Code
0,Afghanistan,AFG
1,Albania,ALB
2,Algeria,DZA
3,American Samoa,ASM
4,Andorra,AND


In [252]:
#define a function to select rows and columns
def data_selection(df):
    #merge the data frame with country list data frame by county Code, so we can drop irrelevant rows
    df = pd.merge(country_list, df, left_on = "Country Code", right_on = "Country Code", how="left")
    df = df.set_index("Country Name")
    #select columns from year 2007 to 2017
    df_new = pd.concat([df["Country Code"],df.loc[:,"2007":"2017"]], axis=1)
    #replace missing value by 0
    df_new = df_new.fillna(0)
    return df_new

In [253]:
#use "data_selection" function to select inbound tourists data
inbound_tourists_clean = data_selection(inbound_tourists)
inbound_tourists_clean.head()

,Country Code,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Country Name,,,,,,,,,,,,
Afghanistan,AFG,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Albania,ALB,1062000.00,1247000.00,1711000.00,2191000.00,2469000.00,3156000.00,2857000.00,3341000.00,3784000.00,4070000.00,4643000.00
Algeria,DZA,1743000.00,1772000.00,1912000.00,2070000.00,2395000.00,2634000.00,2733000.00,2301000.00,1710000.00,2039000.00,2451000.00
American Samoa,ASM,24100.00,24300.00,23500.00,23100.00,22600.00,22600.00,20800.00,21600.00,20300.00,20100.00,20000.00
Andorra,AND,2189000.00,2059000.00,1830000.00,1808000.00,2242000.00,2238000.00,2328000.00,2363000.00,2663000.00,2831000.00,0.00


In [254]:
#define function to reshape the data frame
def reshape_data(df):
    df_new = df[:] 
    df_new.set_index(["Country Code"], inplace = True, append = True)
    #use stack to change the data frame shape
    df_new = df_new.stack()
    df_new = pd.DataFrame(df_new)
    df_new = df_new.reset_index()
    #rename column names
    df_new = df_new.rename(columns = {"level_2":"Year",0:"Amount"})
    return df_new

In [255]:
#use "reshape_data" function to reshape the inbound_touists_clean data frame
inbound_tourists_stack = reshape_data(inbound_tourists_clean)
inbound_tourists_stack.head()

,Country Name,Country Code,Year,Amount
0,Afghanistan,AFG,2007,0.00
1,Afghanistan,AFG,2008,0.00
2,Afghanistan,AFG,2009,0.00
3,Afghanistan,AFG,2010,0.00
4,Afghanistan,AFG,2011,0.00


In [256]:
#make the map plot of the inbound tourists use plotly_express package
inbound_map = px.choropleth(inbound_tourists_stack, 
                            locations="Country Code", 
                            color="Amount", 
                            hover_name="Country Name",
                            #set the animation slide bar
                            animation_frame="Year",
                            color_continuous_scale=px.colors.sequential.BuGn,
                            #set the map type
                            projection="natural earth")
inbound_map
#plotly.offline.plot(inbound_map, filename="inbound_map")

# Top countries that attract tourists

In [257]:
#find the top countries that attract tourists
top_country_inbound = inbound_tourists_clean.sort_values("2017", ascending=False).head(10)
top_inbound_country_list = list(top_country_inbound.index)
top_inbound_country_list

['France',
 'Spain',
 'United States',
 'China',
 'Italy',
 'Mexico',
 'United Kingdom',
 'Turkey',
 'Germany',
 'Thailand']

In [258]:
#select the top countries' data 
top_inbound_data = inbound_tourists_stack.loc[inbound_tourists_stack["Country Name"].isin(top_inbound_country_list)]
top_inbound_data.head()

,Country Name,Country Code,Year,Amount
484,China,CHN,2007,54720000.00
485,China,CHN,2008,53049000.00
486,China,CHN,2009,50875000.00
487,China,CHN,2010,55664000.00
488,China,CHN,2011,57581000.00


In [259]:
#make the line plot for top countries
inbound_top = px.line(top_inbound_data, 
                      x="Year", 
                      y="Amount", 
                      color="Country Name", 
                      line_group="Country Name",  
                      line_shape="linear", 
                      title="Top countries that attract tourists")
inbound_top
#plotly.offline.plot(inbound_top, filename="inbound_top")

In [260]:
top_country_inbound

,Country Code,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Country Name,,,,,,,,,,,,
France,FRA,80853000.00,79218000.00,76764000.00,76647000.00,80499000.00,81980000.00,83634000.00,83701000.00,84452000.00,82682000.00,86861000.00
Spain,ESP,58666000.00,57192000.00,52178000.00,52677000.00,56177000.00,57464000.00,60675000.00,64939000.00,68175000.00,75315000.00,81786000.00
United States,USA,56135000.00,58007000.00,55103000.00,60010000.00,62821000.00,66657000.00,69995000.00,75022000.00,77774000.00,76407000.00,76941000.00
China,CHN,54720000.00,53049000.00,50875000.00,55664000.00,57581000.00,57725000.00,55686000.00,55622000.00,56886000.00,59270000.00,60740000.00
Italy,ITA,43654000.00,42734000.00,43239000.00,43626000.00,46119000.00,46360000.00,47704000.00,48576000.00,50732000.00,52372000.00,58253000.00
Mexico,MEX,21606000.00,22931000.00,22346000.00,23290000.00,23403000.00,23403000.00,24151000.00,29346000.00,32093000.00,35079000.00,39291000.00
United Kingdom,GBR,30870000.00,30142000.00,28199000.00,28295000.00,29306000.00,29282000.00,31063000.00,32613000.00,34436000.00,35814000.00,37651000.00
Turkey,TUR,26122000.00,29792000.00,30187000.00,31364000.00,34654000.00,35698000.00,37795000.00,39811000.00,39478000.00,30289000.00,37601000.00
Germany,DEU,24421000.00,24884000.00,24220000.00,26875000.00,28374000.00,30411000.00,31545000.00,32999000.00,34970000.00,35555000.00,37452000.00


In [261]:
# define fucntion to calculate the yearly military spending growth 
def growth_inbound(df):
    for year in range(2007, 2017,1):
        # add growth columns 
        df["growth"+ str(year+1)] = (df[str(year+1)] - df[str(year)])/df[str(year)]*100
        df_new = df.iloc[:, -10:]
    return df_new

In [262]:
growth_inbound_data = growth_inbound(top_country_inbound)
growth_inbound_data = growth_inbound_data.stack()
growth_inbound_data = growth_inbound_data.reset_index()
growth_inbound_data = growth_inbound_data.rename(columns = {"level_1":"Year",0:"Amount"})
growth_inbound_data.head()

,Country Name,Year,Amount
0,France,growth2008,-2.02
1,France,growth2009,-3.10
2,France,growth2010,-0.15
3,France,growth2011,5.03
4,France,growth2012,1.84


In [263]:
#make the bar plot of the country's inbound tourists growth
growth_bar = px.bar(growth_inbound_data, 
                         x="Year", 
                         y="Amount", 
                         color="Country Name", 
                         hover_name="Country Name",
                         #make grouped bar code
                         barmode="group",
                         title="Top attractive countries inbound tourists yearly growth rate")
growth_bar
#plotly.offline.plot(growth_bar, filename="growth_bar")

# Top countries with the highest receipts from tourists 

In [264]:
#import tourists receipts data set
receipts = pd.read_excel("travel_data.xls", sheet_name="receipts for travel items", header=3, index_col=0)
receipts.head()

,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Country Name,,,,,,,,,,,,,,,,,,,,,
Aruba,ABW,"International tourism, receipts for travel ite...",ST.INT.TVLR.CD,nan,nan,nan,nan,nan,nan,nan,...,1222000000.00,1251000000.00,1351000000.00,1402000000.00,1495000000.00,1605000000.00,1655000000.00,1625000000.00,1731000000.00,nan
Afghanistan,AFG,"International tourism, receipts for travel ite...",ST.INT.TVLR.CD,nan,nan,nan,nan,nan,nan,nan,...,36000000.00,75000000.00,85000000.00,102000000.00,138000000.00,109000000.00,79000000.00,49000000.00,2000000.00,nan
Angola,AGO,"International tourism, receipts for travel ite...",ST.INT.TVLR.CD,nan,nan,nan,nan,nan,nan,nan,...,534000000.00,719000000.00,646000000.00,706000000.00,1234000000.00,1589000000.00,1163000000.00,623000000.00,880000000.00,nan
Albania,ALB,"International tourism, receipts for travel ite...",ST.INT.TVLR.CD,nan,nan,nan,nan,nan,nan,nan,...,1827000000.00,1611000000.00,1631000000.00,1464000000.00,1473000000.00,1699000000.00,1499000000.00,1692000000.00,1942000000.00,nan
Andorra,AND,"International tourism, receipts for travel ite...",ST.INT.TVLR.CD,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [265]:
#use "data_selection" function to select inbound tourists data
receipts_clean = data_selection(receipts)
receipts_clean.head()

,Country Code,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Country Name,,,,,,,,,,,,
Afghanistan,AFG,0.00,8000000.00,36000000.00,75000000.00,85000000.00,102000000.00,138000000.00,109000000.00,79000000.00,49000000.00,2000000.00
Albania,ALB,1378000000.00,1714000000.00,1827000000.00,1611000000.00,1631000000.00,1464000000.00,1473000000.00,1699000000.00,1499000000.00,1692000000.00,1942000000.00
Algeria,DZA,219000000.00,323000000.00,246000000.00,220000000.00,209000000.00,217000000.00,250000000.00,258000000.00,311000000.00,209000000.00,141000000.00
American Samoa,ASM,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,22000000.00,22000000.00
Andorra,AND,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [266]:
#find the top countries with the attract tourists most
top_country_receipts = receipts_clean.sort_values("2017", ascending=False).head(10)
top_receipts_country_list = list(top_country_receipts.index)
top_receipts_country_list

['United States',
 'Spain',
 'France',
 'Thailand',
 'United Kingdom',
 'Italy',
 'Australia',
 'Germany',
 'Macao SAR, China',
 'Japan']

In [267]:
#select the top countries' data 
top_receipts_data = receipts_clean.loc[receipts_clean.index.isin(top_receipts_country_list)]
top_receipts_data.head()

,Country Code,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Country Name,,,,,,,,,,,,
Australia,AUS,21774000000.00,23986000000.00,24566000000.00,28472000000.00,31582000000.00,31557000000.00,30617000000.00,31452000000.00,28892000000.00,34746000000.00,41754000000.00
France,FRA,54210000000.00,57229000000.00,49576000000.00,46466000000.00,55115000000.00,53348000000.00,56457000000.00,58404000000.00,58307000000.00,54690000000.00,61020000000.00
Germany,DEU,36102000000.00,39915000000.00,34724000000.00,34564000000.00,38903000000.00,38068000000.00,41285000000.00,43263000000.00,36883000000.00,37469000000.00,39953000000.00
Italy,ITA,42658000000.00,46191000000.00,40375000000.00,38438000000.00,43241000000.00,40960000000.00,43829000000.00,45547000000.00,39420000000.00,40373000000.00,44548000000.00
Japan,JPN,9345000000.00,10820000000.00,10329000000.00,13224000000.00,11000000000.00,14581000000.00,15093000000.00,18812000000.00,24968000000.00,30752000000.00,34065000000.00


In [268]:
#use "reshape_data" function to reshape the inbound_touists_clean data frame
receipts_clean_stack = reshape_data(top_receipts_data)
receipts_clean_stack.head()

,Country Name,Country Code,Year,Amount
0,Australia,AUS,2007,21774000000.00
1,Australia,AUS,2008,23986000000.00
2,Australia,AUS,2009,24566000000.00
3,Australia,AUS,2010,28472000000.00
4,Australia,AUS,2011,31582000000.00


In [269]:
#make the bar plot of the top countries with highest receipts from trouism
receipts_bar = px.bar(receipts_clean_stack, 
                      x="Year", 
                      y="Amount", 
                      color="Country Name", 
                      hover_name="Country Name",
                      title="Top countries with highest receipts")
receipts_bar
#plotly.offline.plot(receipts_bar, filename="receipts_bar")

# Top countries with the highest tourism share of exports (i.e. tourism-oriented export)

In [270]:
#import receipts count percentage of export data set
percent_export = pd.read_excel("travel_data.xls", sheet_name="receipts (% of total exports)", header=3, index_col=0)
percent_export.head()

,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Country Name,,,,,,,,,,,,,,,,,,,,,
Aruba,ABW,"International tourism, receipts (% of total ex...",ST.INT.RCPT.XP.ZS,nan,nan,nan,nan,nan,nan,nan,...,35.04,68.73,19.79,44.81,69.66,70.66,68.28,69.56,nan,nan
Afghanistan,AFG,"International tourism, receipts (% of total ex...",ST.INT.RCPT.XP.ZS,nan,nan,nan,nan,nan,nan,nan,...,4.19,5.75,5.41,7.21,12.89,6.07,6.14,6.08,1.56,nan
Angola,AGO,"International tourism, receipts (% of total ex...",ST.INT.RCPT.XP.ZS,nan,nan,nan,nan,nan,nan,nan,...,1.34,1.41,0.96,0.99,1.78,2.62,3.40,2.22,2.48,nan
Albania,ALB,"International tourism, receipts (% of total ex...",ST.INT.RCPT.XP.ZS,nan,nan,nan,nan,nan,nan,nan,...,65.77,53.47,48.55,45.66,45.13,49.53,51.98,52.92,49.48,nan
Andorra,AND,"International tourism, receipts (% of total ex...",ST.INT.RCPT.XP.ZS,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [271]:
#use "data_selection" function to select receipts export data
percent_export_clean = data_selection(percent_export)
percent_export_clean.head()

,Country Code,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Country Name,,,,,,,,,,,,
Afghanistan,AFG,0.00,2.63,4.19,5.75,5.41,7.21,12.89,6.07,6.14,6.08,1.56
Albania,ALB,60.01,56.34,65.77,53.47,48.55,45.66,45.13,49.53,51.98,52.92,49.48
Algeria,DZA,0.53,0.58,0.75,0.53,0.39,0.39,0.48,0.50,0.92,0.75,0.46
American Samoa,ASM,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Andorra,AND,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [272]:
#find the top countries with the attract tourists most
top_country_export = percent_export_clean.sort_values("2017", ascending=False).head(10)
top_export_country_list = list(top_country_export.index)
top_export_country_list

['Macao SAR, China',
 'Grenada',
 'Maldives',
 'St. Lucia',
 'Dominica',
 'St. Vincent and the Grenadines',
 'Bahamas, The',
 'Sao Tome and Principe',
 'Antigua and Barbuda',
 'St. Kitts and Nevis']

In [273]:
#use "reshape_data" function to reshape the percent_export_clean data frame
percent_export_stack = reshape_data(percent_export_clean)
percent_export_stack.head()

,Country Name,Country Code,Year,Amount
0,Afghanistan,AFG,2007,0.00
1,Afghanistan,AFG,2008,2.63
2,Afghanistan,AFG,2009,4.19
3,Afghanistan,AFG,2010,5.75
4,Afghanistan,AFG,2011,5.41


In [274]:
#select the top countries' data 
top_country_export = percent_export_stack.loc[percent_export_stack['Country Name'].isin(top_export_country_list)]
top_country_export.head()

,Country Name,Country Code,Year,Amount
77,Antigua and Barbuda,ATG,2007,58.15
78,Antigua and Barbuda,ATG,2008,53.41
79,Antigua and Barbuda,ATG,2009,54.31
80,Antigua and Barbuda,ATG,2010,56.89
81,Antigua and Barbuda,ATG,2011,58.02


In [275]:
#make the polar bar plot for the top countries tourism are export oriented
export_polar = px.bar_polar(top_country_export,
                            #radius column
                            r="Amount",
                            #angle column
                            theta="Country Name",
                            color="Country Name", 
                            #slider animation column
                            animation_frame="Year", 
                            title="Top countries ranked by tourism share of exports")
export_polar
#plotly.offline.plot(export_polar, filename="export_polar")

# GDP and tourists number correlation

In [276]:
#import countries GDP data set
gdp = pd.read_excel("travel_data.xls", sheet_name="GDP", header=3, index_col=0)
gdp.head()

,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Country Name,,,,,,,,,,,,,,,,,,,,,
Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,nan,nan,nan,nan,nan,nan,nan,...,2498882681.56,2390502793.30,2549720670.39,2534636871.51,2581564245.81,2649720670.39,2691620111.73,2646927374.30,2700558659.22,nan
Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,537777811.11,548888895.56,546666677.78,751111191.11,800000044.44,1006666637.78,1399999966.67,...,12439087076.77,15856574731.44,17804292964.10,19907317065.67,20561069558.22,20484885119.73,19907111418.99,19046357714.49,19543976895.42,nan
Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,nan,nan,nan,nan,nan,nan,nan,...,70307163678.19,83799496611.60,111789686464.26,128052853643.45,136709862831.31,145712200312.51,116193649124.48,101123851090.47,122123822333.73,nan
Albania,ALB,GDP (current US$),NY.GDP.MKTP.CD,nan,nan,nan,nan,nan,nan,nan,...,12044208085.86,11926957254.63,12890866742.65,12319784886.20,12776280961.16,13228247844.12,11386931489.80,11883682170.82,13038538300.26,nan
Andorra,AND,GDP (current US$),NY.GDP.MKTP.CD,nan,nan,nan,nan,nan,nan,nan,...,3660530702.97,3355695364.24,3442062830.14,3164615186.95,3281585236.33,3350736367.25,2811489408.89,2877311946.90,3012914131.17,nan


In [277]:
#select relevant years data
gdp = gdp.loc[:,"2007":"2017"]
#reshape the data frame
gdp_data = pd.DataFrame(gdp.stack()).reset_index()
#rename the column name
gdp_data = gdp_data.rename(columns = {"level_1":"Year", 0:"GDP"})
gdp_data.head()

,Country Name,Year,GDP
0,Aruba,2007,2615083798.88
1,Aruba,2008,2745251396.65
2,Aruba,2009,2498882681.56
3,Aruba,2010,2390502793.30
4,Aruba,2011,2549720670.39


In [278]:
#merge the inbound tourists data and gdp data by Country name and year
gdp_inbound = inbound_tourists_stack.merge(gdp_data, left_on=("Country Name","Year"), right_on=("Country Name","Year"))
gdp_inbound.head()

,Country Name,Country Code,Year,Amount,GDP
0,Afghanistan,AFG,2007,0.00,9747879531.86
1,Afghanistan,AFG,2008,0.00,10109225813.67
2,Afghanistan,AFG,2009,0.00,12439087076.77
3,Afghanistan,AFG,2010,0.00,15856574731.44
4,Afghanistan,AFG,2011,0.00,17804292964.10


In [279]:
#make the scatter plot check the relation between GDP and inbound tourists number
gdp_inbound_plot = px.scatter(gdp_inbound, 
                              x="Amount", 
                              y="GDP",
                              #set the color of the plot
                              color_discrete_sequence = px.colors.qualitative.Vivid,
                              hover_name="Country Name",
                              #use log data to plot
                              log_x=True, 
                              log_y=True, 
                              labels="Amount(number of people)",
                              title="Correlation between GDP and number of inbound tourists")
gdp_inbound_plot
#plotly.offline.plot(gdp_inbound_plot, filename="gdp_inbound_plot")

# world tourists travel outbound 

In [280]:
#import tourists outbound data set
outbound_tourists = pd.read_excel("travel_data.xls", sheet_name="number_of_departure", header=3, index_col=0)
outbound_tourists.head()

,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Country Name,,,,,,,,,,,,,,,,,,,,,
Aruba,ABW,"International tourism, number of departures",ST.INT.DPRT,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Afghanistan,AFG,"International tourism, number of departures",ST.INT.DPRT,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Angola,AGO,"International tourism, number of departures",ST.INT.DPRT,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Albania,ALB,"International tourism, number of departures",ST.INT.DPRT,nan,nan,nan,nan,nan,nan,nan,...,3404000.00,3443000.00,4120000.00,3959000.00,3928000.00,4146000.00,4504000.00,4852000.00,5186000.00,nan
Andorra,AND,"International tourism, number of departures",ST.INT.DPRT,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [281]:
#use "data_selection" function to select inbound tourists data
outbound_tourists_clean = data_selection(outbound_tourists)
outbound_tourists_clean.head()

,Country Code,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Country Name,,,,,,,,,,,,
Afghanistan,AFG,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Albania,ALB,2979000.00,3716000.00,3404000.00,3443000.00,4120000.00,3959000.00,3928000.00,4146000.00,4504000.00,4852000.00,5186000.00
Algeria,DZA,1499000.00,1539000.00,1677000.00,1757000.00,1715000.00,1911000.00,2136000.00,2839000.00,3638000.00,4530000.00,5058000.00
American Samoa,ASM,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Andorra,AND,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [282]:
#use "reshape_data" function to reshape the outbound_tourists_clean data frame
outbound_tourists_stack = reshape_data(outbound_tourists_clean)
outbound_tourists_stack.head()

,Country Name,Country Code,Year,Amount
0,Afghanistan,AFG,2007,0.00
1,Afghanistan,AFG,2008,0.00
2,Afghanistan,AFG,2009,0.00
3,Afghanistan,AFG,2010,0.00
4,Afghanistan,AFG,2011,0.00


In [283]:
#make the map plot of world outbound tourist
outbound_map = px.scatter_geo(outbound_tourists_stack, 
                              locations="Country Code", 
                              color="Amount", 
                              hover_name="Country Name", 
                              size="Amount",
                              #set animation column
                              animation_frame="Year",
                              color_continuous_scale=px.colors.sequential.Aggrnyl,
                              projection="natural earth")
outbound_map
#plotly.offline.plot(outbound_map, filename="outbound_map")

In [284]:
#find the top countries with the attract tourists most
top_country_outbound = outbound_tourists_clean.sort_values("2017", ascending=False).head(10)
top_outbound_country_list = list(top_country_outbound.index)
top_outbound_country_list 

['China',
 'Germany',
 'Hong Kong SAR, China',
 'United States',
 'United Kingdom',
 'Poland',
 'Russian Federation',
 'Canada',
 'Italy',
 'France']

In [285]:
top_outbound_data = outbound_tourists_stack.loc[outbound_tourists_stack['Country Name'].isin(top_outbound_country_list)]
top_outbound_data.head()

,Country Name,Country Code,Year,Amount
418,Canada,CAN,2007,25163000.00
419,Canada,CAN,2008,27034000.00
420,Canada,CAN,2009,26204000.00
421,Canada,CAN,2010,28680000.00
422,Canada,CAN,2011,30450000.00


In [286]:
#make the line plot for top outbound countries
outbound_top = px.line(top_outbound_data,
                       #x axis column
                       x="Year",
                       #y axis column
                       y="Amount", 
                       color="Country Name", 
                       line_group="Country Name", 
                       hover_name="Country Name",
                       #line type
                       line_shape="linear",
                       title="Top countries where people like to travel")
outbound_top
#plotly.offline.plot(outbound_top, filename="outbound_top")

# Relationship between population and the number of outbound tourists

In [287]:
#import the population data set
population = pd.read_excel("travel_data.xls", sheet_name="population", header=3, index_col=0)
population.head()

,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Country Name,,,,,,,,,,,,,,,,,,,,,
Aruba,ABW,"Population, total",SP.POP.TOTL,54211.00,55438.00,56225.00,56695.00,57032.00,57360.00,57715.00,...,101453.00,101669.00,102053.00,102577.00,103187.00,103795.00,104341.00,104822.00,105264.00,nan
Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996351.00,9166764.00,9345868.00,9533954.00,9731361.00,9938414.00,10152331.00,...,28004331.00,28803167.00,29708599.00,30696958.00,31731688.00,32758020.00,33736494.00,34656032.00,35530081.00,nan
Angola,AGO,"Population, total",SP.POP.TOTL,5643182.00,5753024.00,5866061.00,5980417.00,6093321.00,6203299.00,6309770.00,...,22549547.00,23369131.00,24218565.00,25096150.00,25998340.00,26920466.00,27859305.00,28813463.00,29784193.00,nan
Albania,ALB,"Population, total",SP.POP.TOTL,1608800.00,1659800.00,1711319.00,1762621.00,1814135.00,1864791.00,1914573.00,...,2927519.00,2913021.00,2905195.00,2900401.00,2895092.00,2889104.00,2880703.00,2876101.00,2873457.00,nan
Andorra,AND,"Population, total",SP.POP.TOTL,13411.00,14375.00,15370.00,16412.00,17469.00,18549.00,19647.00,...,84462.00,84449.00,83751.00,82431.00,80788.00,79223.00,78014.00,77281.00,76965.00,nan


In [288]:
#use "data_selection" function to select population data set
population_clean = data_selection(population)
population_clean.head()

,Country Code,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Country Name,,,,,,,,,,,,
Afghanistan,AFG,26616792.00,27294031.00,28004331.00,28803167.00,29708599.00,30696958.00,31731688.00,32758020.00,33736494.00,34656032.00,35530081.00
Albania,ALB,2970017.00,2947314.00,2927519.00,2913021.00,2905195.00,2900401.00,2895092.00,2889104.00,2880703.00,2876101.00,2873457.00
Algeria,DZA,34300076.00,34860715.00,35465760.00,36117637.00,36819558.00,37565847.00,38338562.00,39113313.00,39871528.00,40606052.00,41318142.00
American Samoa,ASM,57903.00,57030.00,56227.00,55637.00,55320.00,55230.00,55307.00,55437.00,55537.00,55599.00,55641.00
Andorra,AND,82683.00,83861.00,84462.00,84449.00,83751.00,82431.00,80788.00,79223.00,78014.00,77281.00,76965.00


In [289]:
#def a function concate the population data with outbound tourists data
def population_outbound(year):
    df1 = population_clean.loc[population_clean.index.isin(top_outbound_country_list)].loc[:,year]
    df2 = outbound_tourists_clean.loc[outbound_tourists_clean.index.isin(top_outbound_country_list)].loc[:,year]
    df = pd.concat([df1, df2], axis=1)
    return df

In [290]:
population_outbound("2017")

,2017,2017
Country Name,,
Canada,36708083.00,33060000.00
China,1386395000.00,143035000.00
France,67105513.00,29055000.00
Germany,82685827.00,92402000.00
"Hong Kong SAR, China",7391700.00,91304000.00
Italy,60536709.00,31805000.00
Poland,37974826.00,46700000.00
Russian Federation,144496740.00,39629000.00
United Kingdom,66023290.00,74189000.00


In [291]:
#make the line-bar plot
#make the scatter plot
trace1=go.Scatter(
    x=population_outbound("2017").index,
    y=population_outbound("2017").iloc[:,0],
    name="population")
#make the bar plot
trace2=go.Bar(
    x=population_outbound("2017").index,
    y=population_outbound("2017").iloc[:,1],
    name="tourists",
    yaxis="y2",
    #transparent level
    opacity=0.6,
    #bar wide
    width=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])
data=[trace1,trace2]
layout=go.Layout(
    title="Outbound tourists number vs country's population",
    yaxis=dict(title="population"),
    yaxis2=dict(title="number of people", 
                titlefont=dict(color="rgb(148, 103, 189)"),
                tickfont=dict(color="rgb(148, 103, 189)"),
                overlaying="y",
                side="right"))
outbound_population_compare=go.Figure(data=data, layout=layout)
py.iplot(outbound_population_compare)
#plotly.offline.plot(outbound_population_compare, filename="outbound_population_compare")

In [292]:
#merge the outbound tourists data and gdp data
gdp_outbound = outbound_tourists_stack.merge(gdp_data, left_on=("Country Name","Year"), right_on=("Country Name","Year"))
gdp_outbound.head()

,Country Name,Country Code,Year,Amount,GDP
0,Afghanistan,AFG,2007,0.00,9747879531.86
1,Afghanistan,AFG,2008,0.00,10109225813.67
2,Afghanistan,AFG,2009,0.00,12439087076.77
3,Afghanistan,AFG,2010,0.00,15856574731.44
4,Afghanistan,AFG,2011,0.00,17804292964.10


In [293]:
#make the scatter plot to check the relationship between tourists number and GDP
gdp_outbound_plot = px.scatter(gdp_outbound, 
                               x="Amount", 
                               y="GDP",  
                               hover_name="Country Name",
                               #use log data to make the plot
                               log_x=True, 
                               log_y=True,
                               title="Correlation between GDP and outbound tourists")
gdp_outbound_plot
#plotly.offline.plot(gdp_outbound_plot, filename="gdp_outbound_plot")

In [294]:
#import expenditure data set
expenditure = pd.read_excel("travel_data.xls", sheet_name="expenditure for travel item", header=3, index_col=0)
expenditure_clean = data_selection(expenditure)
expenditure_clean.head()

,Country Code,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Country Name,,,,,,,,,,,,
Afghanistan,AFG,0.00,26000000.00,68000000.00,103000000.00,246000000.00,105000000.00,132000000.00,137000000.00,147000000.00,79000000.00,118000000.00
Albania,ALB,1268000000.00,1555000000.00,1584000000.00,1360000000.00,1564000000.00,1284000000.00,1479000000.00,1590000000.00,1237000000.00,1262000000.00,1432000000.00
Algeria,DZA,376000000.00,468000000.00,456000000.00,601000000.00,527000000.00,529000000.00,471000000.00,608000000.00,685000000.00,474000000.00,580000000.00
American Samoa,ASM,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Andorra,AND,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [310]:
expenditure_p = expenditure_clean.iloc[:,1:12]/outbound_tourists_clean.iloc[:,1:12]
expenditure_p = expenditure_p[~expenditure_p.isin([np.nan, np.inf, -np.inf]).any(1)]
expenditure_p.head()

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Country Name,,,,,,,,,,,
Albania,425.65,418.46,465.33,395.00,379.61,324.32,376.53,383.50,274.64,260.10,276.13
Algeria,250.83,304.09,271.91,342.06,307.29,276.82,220.51,214.16,188.29,104.64,114.67
Argentina,944.32,993.93,910.66,932.92,847.59,841.87,879.34,918.06,894.33,864.06,857.07
Armenia,809.83,843.02,842.21,1028.42,960.84,832.12,858.73,854.76,839.09,844.81,856.95
Australia,3012.45,3377.58,2985.66,3175.84,3514.51,3419.14,3156.76,2839.56,2420.11,2701.73,3133.10


In [318]:
#select the top countries' data 
top_expenditure_p_data = expenditure_p.loc[expenditure_p.index.isin(top_outbound_country_list)].sort_values("2017", ascending=False)
top_expenditure_p_data.head()

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Country Name,,,,,,,,,,,
China,727.30,788.70,917.03,956.33,1033.24,1225.95,1309.53,1949.89,1953.94,1850.90,1801.89
United States,1393.23,1453.91,1310.49,1418.63,1514.99,1653.08,1599.49,1549.73,1543.99,1540.26,1539.55
France,1348.79,1604.21,1470.21,1529.65,1708.85,1574.08,1618.68,1746.55,1482.59,1522.60,1431.77
Germany,1012.87,1062.61,951.54,903.44,1017.40,1006.12,1044.15,1123.41,924.86,878.42,970.51
United Kingdom,1102.72,1191.72,1098.68,1130.09,1106.09,1149.37,1151.46,1186.48,1126.14,1033.07,966.06


In [319]:
#reshape the top_expenditure_p_data data frame
top_expenditure_p_stack = top_expenditure_p_data.stack().reset_index()
top_expenditure_p_stack = top_expenditure_p_stack.rename(columns = {"level_1":"Year",0:"Per capita expenditure"})
top_expenditure_p_stack.head()

,Country Name,Year,Per capita expenditure
0,China,2007,727.30
1,China,2008,788.70
2,China,2009,917.03
3,China,2010,956.33
4,China,2011,1033.24


In [322]:
#make the bar plot of the top countries' people with highest spending in the travel
expenditure_bar = px.bar(top_expenditure_p_stack, 
                         x="Year", 
                         y="Per capita expenditure", 
                         color="Country Name", 
                         hover_name="Country Name",
                         title="Top countries per capita expenditure in travel(exclude international transportation)")
expenditure_bar
#plotly.offline.plot(expenditure_bar, filename="expenditure_bar")